In [2]:
import pandas as pd
import numpy as np

# 读取训练集和测试集
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

# step1：查看数据的基本信息
print '训练集:-----------------：\n', train_data.info()
print '测试集:-----------------\n', test_data.info()


训练集:-----------------：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
测试集:-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418

In [6]:

# step2：特征选择
selected_features = ['Pclass', 'Sex', 'Age', 'Embarked', 'SibSp', 'Parch', 'Fare']

x_train = train_data[selected_features]
x_test = test_data[selected_features]
y_train = train_data['Survived']

print x_train['Embarked'].value_counts()
print x_test['Embarked'].value_counts()


S    644
C    168
Q     77
Name: Embarked, dtype: int64
S    270
C    102
Q     46
Name: Embarked, dtype: int64


In [7]:

# step3：数据预处理
# 使用出现频率最高的 S填充Embarked的缺省值
x_train['Embarked'].fillna('S', inplace=True)
x_test['Embarked'].fillna('S', inplace=True)

# 使用品均值填充Age
x_train['Age'].fillna(x_train['Age'].mean(), inplace=True)
x_test['Age'].fillna(x_test['Age'].mean(), inplace=True)
x_test['Fare'].fillna(x_test['Fare'].mean(), inplace=True)
print  '查看处理后数据:-----------------'
print x_train.info()
print x_test.info()

查看处理后数据:-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
Embarked    891 non-null object
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         418 non-null float64
Embarked    418 non-null object
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB
None


/Users/baixiaoling/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [8]:

# step4：特征向量化
from sklearn.feature_extraction import DictVectorizer

dict_vec = DictVectorizer(sparse=False)
x_train = dict_vec.fit_transform(x_train.to_dict(orient='record'))
print "特征向量化结束后：:-----------------",dict_vec.feature_names_
x_test = dict_vec.transform(x_test.to_dict(orient='record'))

特征向量化结束后：:----------------- ['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


In [9]:

# step5: 导入模型训练
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

# step6: 进行预测
# 使用5折交叉验证对rfc进行评估
from sklearn.model_selection import cross_val_score

print cross_val_score(rfc, x_train, y_train, cv=5).mean()

rfc.fit(x_train, y_train)

rfc_y_prediction = rfc.predict(x_test)
rfc_submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': rfc_y_prediction})
rfc_submission.to_csv('./gender_submission.csv', index=False)

0.796953564223


In [ ]:
from XGBoo